In [1]:
import scanpy as sc
import os
import anndata as ad
import cellrank as cr
import numpy as np
import scipy as sp

2023-10-20 13:36:58.859211: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-20 13:36:59.479302: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-20 13:37:00.811423: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-20 13:37:00.812530: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 13:37:02.887126: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [14]:
# DATA_PATH="../../data/simulated/Synthetic_1000/dyn-SW/dyn-SW-1000-2/"
# DATA_PATH="../../data/simulated/Synthetic_1000/dyn-LI/dyn-LI-1000-2/"
# DATA_PATH="../../data/simulated/Synthetic_1000/dyn-CY/dyn-CY-1000-2/"
DATA_PATH="/home/stephenz/stephenz/locaTE-paper/data/simulated/Curated/GSD/GSD-1000-2/"

In [15]:
adata = ad.read_h5ad(os.path.join(DATA_PATH, "anndata.h5ad"))

In [16]:
k = adata.uns['neighbors']['params']['n_neighbors']

In [17]:
del adata.uns['neighbors']

In [18]:
# dpt is true time, dpt_pseudotime is diffusion pseudotime
adata.obsm["X_dpt"] = adata.obs["dpt"].to_numpy().reshape(-1, 1)

In [19]:
sc.pp.neighbors(adata, k, use_rep = "X_dpt")

In [20]:
def compute_velocity_kernel(k, sigma):
    # compute velocity-induced transition matrix with kernel k and bandwidth sigma
    if sigma is None:
        # median heuristic, see e.g. https://github.com/theislab/cellrank/blob/f6d281e62a5cee976d25973b5a5e4c1050f37429/cellrank/kernels/_velocity_kernel.py#L230 
        logits_all = np.concatenate([k(adata.obsm["v"].iloc[i, :].to_numpy().reshape(1, -1).astype(np.float64),
                        (adata.X[adata.uns['neighbors']['connectivities'][i, :].indices, :] - adata.X[i, :]).astype(np.float64),
                                softmax_scale =  1.0)[1] for i in range(adata.shape[0])])
        sigma = 1/np.median(logits_all)
    probs_all_fwd = [k(adata.obsm["v"].iloc[i, :].to_numpy().reshape(1, -1).astype(np.float64),
                    (adata.X[adata.uns['neighbors']['connectivities'][i, :].indices, :] - adata.X[i, :]).astype(np.float64),
                    softmax_scale = sigma)[0] for i in range(adata.shape[0])]
    P_velo = sp.sparse.csr_matrix(adata.uns['neighbors']['connectivities'].shape)
    for i, p in enumerate(probs_all_fwd):
        P_velo[i, adata.uns['neighbors']['connectivities'][i, :].indices] = p
    return P_velo

# apply velocity kernels 
for (k, sigma, kerneltype) in zip([cr.tl.kernels.DotProductScheme(), cr.tl.kernels.CorrelationScheme(), cr.tl.kernels.CosineScheme()],
                            [None, None, None], ["dot", "corr", "cos"]):
    adata.obsm["P_velo_%s" % kerneltype] = compute_velocity_kernel(k, sigma)
# diffusion pseudotime kernel
k = cr.tl.kernels.PseudotimeKernel(adata, time_key = "dpt_pseudotime").compute_transition_matrix(threshold_scheme = "soft") 
adata.obsm["P_dpt"] = k.transition_matrix

  0%|          | 0/1000 [00:00<?, ?cell/s]

In [21]:
adata.obsm["P_velo_dot"][0, :]

<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 24 stored elements in Compressed Sparse Row format>

In [22]:
# save transition matrices
for k in [x for x in adata.obsm.keys() if "P_" in x]:
    print("Writing transition matrix %s..." % k)
    try:
        np.save(os.path.join(DATA_PATH, f"{k}_dpt_knn.npy"), np.array(adata.obsm[k].todense()))
    except:
        np.save(os.path.join(DATA_PATH, f"{k}_dpt_knn.npy"), np.array(adata.obsm[k]))

Writing transition matrix P_dpt...
Writing transition matrix P_statot...
Writing transition matrix P_statot_ent...
Writing transition matrix P_velo_corr...
Writing transition matrix P_velo_cos...
Writing transition matrix P_velo_dot...
